In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
#Boilerplate code
import os, json
from glob import glob

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model,load_model,Sequential

from tensorflow.keras.layers import *
from tensorflow.keras import backend as K

import tensorflow as tf

tf.VERSION

'1.13.1'

In [3]:
#Download the files from the Kaggle NSDC page
!wget -qq https://www.dropbox.com/s/dlpf8lirpf69nvb/ndsc-beginner.zip

#Unzip the zip file
!unzip -qq ndsc-beginner.zip

#Remove zipfile, we won't need it anymore
!rm ndsc-beginner.zip

#Check directory contents
!ls

#The files we want are all in the folder ndsc-beginner

replace ndsc-beginner/categories.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
__MACOSX  ndsc-beginner  sample_data


In [4]:
#Now let's import train.csv using pandas

import pandas as pd

df = pd.read_csv('./ndsc-beginner/train.csv')
print(df)

            itemid                                              title  \
0           307504               nyx sex bomb pallete natural palette   
1           461203  etude house precious mineral any cushion pearl...   
2          3592295                           milani rose powder blush   
3          4460167                etude house baby sweet sugar powder   
4          5853995       bedak revlon color stay aqua mineral make up   
5          6208490                            dr pure whitening cream   
6          6959546                         chanel powder blush malice   
7          8984461                     snail white cream original 100   
8          9184082                          sunprise all proof spf 50   
9         10345338             eyebrow powder nyx satuan rp 15.000 pc   
10        11016411                        monistat chafing relief gel   
11        11425757                       milani rose powder blush tea   
12        13017527                        the balm 

In [33]:
import numpy as np
import pandas as pd

#sort items by category 
df=df.sort_values(by='Category',ascending=True)
cat_list=list(range(0,58))

#divide the dataset into 80% training, 20% validation
train_set_percentage = .8
total_train_data=pd.DataFrame()
total_val_data=pd.DataFrame()

#create new dataframe for each category with all the details of the product inside
for i in range(0,len(cat_list)):
  cat_list[i]=pd.DataFrame()
  cat_list[i]=df[(df['Category']==i)]
  #dog = 'sub_cat_list_{}'.format(i)
  #cat_list[i].to_csv(dog+'.csv',index=False)
#print('exported successfully')
#print(cat_list[46])
  train_df = cat_list[i][:int(len(cat_list[i])*train_set_percentage)]
  val_df = cat_list[i][int(len(cat_list[i])*train_set_percentage):] 
  total_train_data = total_train_data.append(train_df,ignore_index =True)
  total_val_data = total_val_data.append(val_df,ignore_index =True)
  total_train_data = total_train_data.sample(frac=1).reset_index(drop=True)
  total_val_data = total_val_data.sample(frac=1).reset_index(drop=True)
print(total_train_data)
# train_fashion_df = df2[:int(len(fashion_df)*train_set_percentage)]
# val_fashion_df = df2[int(len(fashion_df)*train_set_percentage):]

# train_beauty_df = df3[:int(len(df3)*train_set_percentage)]
# val_beauty_df = df3[int(len(beauty_df)*train_set_percentage):]

# train_df4 = df4[:int(len(df4)*train_set_percentage)]
# val_df4 = df4[int(len(df4)*train_set_percentage):]

# train_df5 = df5[:int(len(df5)*train_set_percentage)]
# val_df5 = df5[int(len(df5)*train_set_percentage):]
# df_new_train = pd.concat([train_df, train_df2, train_df3, train_df4, train_df5])
# df_new_val = pd.concat([val_df, val_df2, val_df3, val_df4, val_df5])


# Shuffle
# df = df_new_train.sample(frac=1).reset_index(drop=True)
# df_val = df_new_val.sample(frac=1).reset_index(drop=True)


            itemid                                              title  \
0           307504               nyx sex bomb pallete natural palette   
1       1466472007  kylie holiday edition box silver make up set p...   
2       1480795815         sale la tulipe peeling cream 250gr make up   
3       1776570859   makeup kit book the balm pretty color rose shade   
4        933517183  12 color contour palette kiss beauty 92 93 94 ...   
5       1086032809  original defect in the balm of your hand vol.1...   
6       1581989893  terjamin lt pro translucent powder 30 g 100%or...   
7       1129898509          hot sale nyx eyebrow cake powder brunette   
8       1618394616  termurah grosir paket kosmetik wardah eyeshado...   
9       1184154835  magical 3pcs stick shimmer+bubuk bronzer kulit...   
10      1195444747    cuci gudang nyx cream highlight contour palette   
11      1803146221             cosrx low ph good morning gel cleanser   
12      1280722433  tahan lama kos1585 mn coutour c